In [1]:
%matplotlib inline
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import numpy as np
tf.set_random_seed(777)

In [2]:
sentence = ("if you want to build aship, don't drup up people together to"
            "collect wood and don't assign them tasks and work, but rather"
            "teach them to long for the endless immensity of the sea.")

seq_length = 10
char_set = list(set(sentence))
char_dic = {w: i for i , w in enumerate(char_set)}

dataX = [] 
dataY = []
for i in range(0,len(sentence) - seq_length) :
    x_str = sentence[i:i+ seq_length]
    y_str = sentence[i+1:i+seq_length +1]
    print(i ,x_str,'->',y_str)
    
    x=[char_dic[c] for c in x_str]
    y=[char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)


batch_size = len(dataX)



0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build as
13 o build as ->  build ash
14  build ash -> build ashi
15 build ashi -> uild aship
16 uild aship -> ild aship,
17 ild aship, -> ld aship, 
18 ld aship,  -> d aship, d
19 d aship, d ->  aship, do
20  aship, do -> aship, don
21 aship, don -> ship, don'
22 ship, don' -> hip, don't
23 hip, don't -> ip, don't 
24 ip, don't  -> p, don't d
25 p, don't d -> , don't dr
26 , don't dr ->  don't dru
27  don't dru -> don't drup
28 don't drup -> on't drup 
29 on't drup  -> n't drup u
30 n't drup u -> 't drup up
31 't drup up -> t drup up 
32 t drup up  ->  drup up p
33  drup up p -> drup up pe
34 drup up pe -> rup up peo
35 rup up peo -> up up peop
36

In [3]:
X = tf.placeholder(tf.int32, [None,seq_length])
Y = tf.placeholder(tf.int32, [None,seq_length])

X_one_hot = tf.one_hot(X,num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size ,state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell]*2,state_is_tuple=True)


In [6]:
outputs, _states = tf.nn.dynamic_rnn(cell,X_one_hot , dtype = tf.float32)

X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

weights =tf.ones([batch_size,seq_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=Y, weights=weights)
loss =tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediection = tf.argmax(outputs,axis=2)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        _, l, results = sess.run(
            [train, loss, outputs], feed_dict={X: dataX, Y: dataY})
        for j, result in enumerate(results):
            index = np.argmax(result, axis=1)
            print(i, j, ''.join([char_set[t] for t in index]), l)
    
    results = sess.run(outputs, feed_dict={X: dataX})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        if j is 0:  # print all for the first result to make a sentence
            print(''.join([char_set[t] for t in index]), end='')
        else:
            print(char_set[index[-1]], end='')


0 0 oooomaaaaa 3.2181866
0 1 unhhnaaaay 3.2181866
0 2 ehfaaaaaaa 3.2181866
0 3 ottaaaaaaa 3.2181866
0 4 taaaaaaaaa 3.2181866
0 5 aaaaayayya 3.2181866
0 6 eomeyeyyee 3.2181866
0 7 ottyyyyaaa 3.2181866
0 8 eyyyyyaaaa 3.2181866
0 9 yyyyyyaaaa 3.2181866
0 10 yhhymyaaay 3.2181866
0 11 ehmmeaaayy 3.2181866
0 12 yytyaaayya 3.2181866
0 13 tffaaayyyi 3.2181866
0 14 eeemykkykk 3.2181866
0 15 ppppkkykkk 3.2181866
0 16 aaayohiiii 3.2181866
0 17 ooooooiio, 3.2181866
0 18 klohheko.. 3.2181866
0 19 ooheeeo..o 3.2181866
0 20 eeeekb...o 3.2181866
0 21 eppkcca.ag 3.2181866
0 22 cbbcc,,,ty 3.2181866
0 23 bbrr.ootyy 3.2181866
0 24 oowwootyyw 3.2181866
0 25 lwwwwyyyww 3.2181866
0 26 gffiyyyyyw 3.2181866
0 27 eooyyyywhh 3.2181866
0 28 ogtyyyyhaa 3.2181866
0 29 ttyyyyyaaa 3.2181866
0 30 yyyyyyyaaa 3.2181866
0 31 uyy'hhhhaa 3.2181866
0 32 yhhhhhhaaa 3.2181866
0 33 eohhaaaaaa 3.2181866
0 34 ooaaaaaaaa 3.2181866
0 35 aaaaaaaaaa 3.2181866
0 36 aaaaaaaaaw 3.2181866
0 37 lwaaawwwww 3.2181866
0 38 eaaaaaawww 3.2181

4 151            2.9160786
4 152            2.9160786
4 153            2.9160786
4 154            2.9160786
4 155            2.9160786
4 156            2.9160786
4 157            2.9160786
4 158            2.9160786
4 159            2.9160786
4 160            2.9160786
4 161            2.9160786
4 162            2.9160786
4 163            2.9160786
4 164            2.9160786
4 165            2.9160786
4 166            2.9160786
5 0            2.8956726
5 1            2.8956726
5 2            2.8956726
5 3            2.8956726
5 4            2.8956726
5 5            2.8956726
5 6            2.8956726
5 7            2.8956726
5 8            2.8956726
5 9            2.8956726
5 10            2.8956726
5 11            2.8956726
5 12            2.8956726
5 13            2.8956726
5 14            2.8956726
5 15            2.8956726
5 16            2.8956726
5 17            2.8956726
5 18            2.8956726
5 19            2.8956726
5 20            2.8956726
5 21            2.8956726
5 22  

9 65            2.8207402
9 66            2.8207402
9 67            2.8207402
9 68            2.8207402
9 69            2.8207402
9 70            2.8207402
9 71            2.8207402
9 72            2.8207402
9 73            2.8207402
9 74            2.8207402
9 75            2.8207402
9 76          n 2.8207402
9 77          n 2.8207402
9 78         nn 2.8207402
9 79        nnn 2.8207402
9 80       nnnn 2.8207402
9 81      nnnnn 2.8207402
9 82     nnnnnn 2.8207402
9 83    nnnnnnn 2.8207402
9 84            2.8207402
9 85            2.8207402
9 86            2.8207402
9 87            2.8207402
9 88          n 2.8207402
9 89         nn 2.8207402
9 90        nnn 2.8207402
9 91         nn 2.8207402
9 92       nnnn 2.8207402
9 93        nnn 2.8207402
9 94       nnnn 2.8207402
9 95      nnnnn 2.8207402
9 96      nnnnn 2.8207402
9 97            2.8207402
9 98            2.8207402
9 99            2.8207402
9 100            2.8207402
9 101            2.8207402
9 102            2.8207402
9 103    

13 147  tt ttsiii 2.5819604
13 148    tttiiin 2.5819604
13 149     tiii   2.5819604
13 150    tiino t 2.5819604
13 151   tssnnott 2.5819604
13 152  tinnnntts 2.5819604
13 153  iiinnntsi 2.5819604
13 154  iinnntsss 2.5819604
13 155  tsnnsssn  2.5819604
13 156  tsssssn   2.5819604
13 157   ttsssh   2.5819604
13 158    tss  t  2.5819604
13 159   tts tt t 2.5819604
13 160   ii tt t  2.5819604
13 161  ii tt t   2.5819604
13 162    tt t    2.5819604
13 163   tt t   t 2.5819604
13 164     t   ts 2.5819604
13 165    t   tss 2.5819604
13 166   t   tssn 2.5819604
14 0   t   t tn 2.5051057
14 1      t tn  2.5051057
14 2 o   t tn t 2.5051057
14 3    t tn te 2.5051057
14 4   t tt t   2.5051057
14 5  t sn te t 2.5051057
14 6 ooss te tt 2.5051057
14 7  tt te tt  2.5051057
14 8 s  t  tt   2.5051057
14 9   te tt    2.5051057
14 10  te tt  t  2.5051057
14 11 oe tt    t 2.5051057
14 12   tt    tt 2.5051057
14 13  tt    tni 2.5051057
14 14 ot  t tsi  2.5051057
14 15    t tti   2.5051057
14 16   t tsi    2

18 62 e tl totd  2.01126
18 63 et  totd t 2.01126
18 64    totd tn 2.01126
18 65   totd tnd 2.01126
18 66 etotd tnd  2.01126
18 67 totd tnd t 2.01126
18 68  t  tnd to 2.01126
18 69    tnd tot 2.01126
18 70   tnd totd 2.01126
18 71  tnd totd  2.01126
18 72 tnd t td   2.01126
18 73 sd t td  t 2.01126
18 74   totd  tn 2.01126
18 75  totd  tns 2.01126
18 76 totd  tnsi 2.01126
18 77  tdd tnsim 2.01126
18 78  ld tnsimn 2.01126
18 79  d tnsimn  2.01126
18 80   tnsimn t 2.01126
18 81 etnsimn th 2.01126
18 82 tnsimn the 2.01126
18 83 siiin thet 2.01126
18 84  imm the   2.01126
18 85  mm the  t 2.01126
18 86  o toe  th 2.01126
18 87   toe  toh 2.01126
18 88  toe  tohs 2.01126
18 89 toe  thhss 2.01126
18 90 ee  totss  2.01126
18 91 e  thhss t 2.01126
18 92  tthhsi tn 2.01126
18 93  tonsi tnd 2.01126
18 94 tohsi tnd  2.01126
18 95 etsi tnd t 2.01126
18 96 sii tnd to 2.01126
18 97    and tot 2.01126
18 98   nnd tott 2.01126
18 99  nnd tott  2.01126
18 100 tnd tott   2.01126
18 101 sd tott  p 2.0112

...

482 60 ollect woo 0.22497436
482 61 nlect wood 0.22497436
482 62 eect wood  0.22497436
482 63 e t wood a 0.22497436
482 64  t wood an 0.22497436
482 65 o wood and 0.22497436
482 66  dood and  0.22497436
482 67 tood and d 0.22497436
482 68 ood and do 0.22497436
482 69 nd and don 0.22497436
482 70 n and don' 0.22497436
482 71  and don't 0.22497436
482 72 tnd don't  0.22497436
482 73 sd don't a 0.22497436
482 74 d don't as 0.22497436
482 75  aon't ass 0.22497436
482 76 ton't assi 0.22497436
482 77  n't assig 0.22497436
482 78 n't assign 0.22497436
482 79 dt assign  0.22497436
482 80 t assign t 0.22497436
482 81  dssign th 0.22497436
482 82 tnsign the 0.22497436
482 83 ssign them 0.22497436
482 84 s gn them  0.22497436
482 85 stn them t 0.22497436
482 86 fn them ta 0.22497436
482 87 e them tas 0.22497436
482 88 dthem task 0.22497436
482 89 toem tasks 0.22497436
482 90  er tasks  0.22497436
482 91 em tasks a 0.22497436
482 92   tasks an 0.22497436
482 93  tasks and 0.22497436
482 94 tosks a

486 142  toe endle 0.22496484
486 143 toemendles 0.22496484
486 144  emendless 0.22496484
486 145 emendless  0.22496484
486 146 rtndless i 0.22496484
486 147 tndless im 0.22496484
486 148 rdless imm 0.22496484
486 149 d ess imme 0.22496484
486 150  ess immen 0.22496484
486 151 e s immens 0.22496484
486 152 rs immensi 0.22496484
486 153   immensit 0.22496484
486 154  immensity 0.22496484
486 155 tmmensity  0.22496484
486 156 gmensity o 0.22496484
486 157  ensity of 0.22496484
486 158  nsity of  0.22496484
486 159 rdity of t 0.22496484
486 160 dity of th 0.22496484
486 161  ty of the 0.22496484
486 162 gy of the  0.22496484
486 163   of the s 0.22496484
486 164 oof the se 0.22496484
486 165 tf the sea 0.22496484
486 166 n the sea. 0.22496484
487 0 f you want 0.22496411
487 1 oyou want  0.22496411
487 2 tou want t 0.22496411
487 3  u want to 0.22496411
487 4 n want to  0.22496411
487 5 pwant to b 0.22496411
487 6 tont to bu 0.22496411
487 7 ont to bui 0.22496411
487 8 sd to buil 0.2249641

490 99 iind work, 0.22495723
490 100 tsd work,  0.22495723
490 101 sd work, b 0.22495723
490 102 d work, bu 0.22495723
490 103  aork, but 0.22495723
490 104 tork, but  0.22495723
490 105 ork, but r 0.22495723
490 106 n , but ra 0.22495723
490 107  , but rat 0.22495723
490 108 , but rath 0.22495723
490 109  dut rathe 0.22495723
490 110 tui rather 0.22495723
490 111 ui rathert 0.22495723
490 112 p ratherte 0.22495723
490 113 haathertea 0.22495723
490 114 tatherteac 0.22495723
490 115  therteach 0.22495723
490 116 sherteach  0.22495723
490 117 hem each t 0.22495723
490 118 em each th 0.22495723
490 119 n each the 0.22495723
490 120  each them 0.22495723
490 121 hach them  0.22495723
490 122 nch them t 0.22495723
490 123 sh them to 0.22495723
490 124 o them to  0.22495723
490 125 ethem to l 0.22495723
490 126 toem to lo 0.22495723
490 127 hem ta lon 0.22495723
490 128 em ta long 0.22495723
490 129 n ta long  0.22495723
490 130  ta long f 0.22495723
490 131 to bong fo 0.22495723
490 132 h l

494 56  thcollect 0.22495286
494 57 to ollect  0.22495286
494 58   ollect w 0.22495286
494 59 nollect wo 0.22495286
494 60 ollect woo 0.22495286
494 61 nlect wood 0.22495286
494 62 eect wood  0.22495286
494 63 e t wood a 0.22495286
494 64  t wood an 0.22495286
494 65 o wood and 0.22495286
494 66  aood and  0.22495286
494 67 tood and d 0.22495286
494 68 ood and do 0.22495286
494 69 nd and don 0.22495286
494 70 n and don' 0.22495286
494 71  and don't 0.22495286
494 72 tnd don't  0.22495286
494 73 nd don't a 0.22495286
494 74 d don't as 0.22495286
494 75  aon't ass 0.22495286
494 76 ton't assi 0.22495286
494 77  n't assig 0.22495286
494 78 n't assign 0.22495286
494 79 dt assign  0.22495286
494 80 t assign t 0.22495286
494 81  assign th 0.22495286
494 82 tnsign the 0.22495286
494 83 nsign them 0.22495286
494 84   gn them  0.22495286
494 85  tn them t 0.22495286
494 86 fn them ta 0.22495286
494 87   them tas 0.22495286
494 88 dthem task 0.22495286
494 89 toem tosks 0.22495286
494 90  em tos

498 34  up up peo 0.22496419
498 35  p up peop 0.22496419
498 36 p up peopl 0.22496419
498 37  up people 0.22496419
498 38 tp people  0.22496419
498 39 p ueople t 0.22496419
498 40  ueople to 0.22496419
498 41 teople tog 0.22496419
498 42  ople toge 0.22496419
498 43 mple toget 0.22496419
498 44 nle togeth 0.22496419
498 45  e togethe 0.22496419
498 46 ectogether 0.22496419
498 47 meogether  0.22496419
498 48 to ether t 0.22496419
498 49 h ether to 0.22496419
498 50 nether toc 0.22496419
498 51 ether toco 0.22496419
498 52 mher tocol 0.22496419
498 53 hem tocoll 0.22496419
498 54 em tocolle 0.22496419
498 55 m tocollec 0.22496419
498 56  thcollect 0.22496419
498 57 to ollect  0.22496419
498 58 h ollect w 0.22496419
498 59 nollect wo 0.22496419
498 60 hllect woo 0.22496419
498 61 nlect wood 0.22496419
498 62 eect wood  0.22496419
498 63 ect wood a 0.22496419
498 64 mt wood an 0.22496419
498 65 h wood and 0.22496419
498 66 haood and  0.22496419
498 67 tord and d 0.22496419
498 68 ood and